In [1]:
import copy
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import random
import time
import sys, io

import mylibrary.datasets as datasets
import mylibrary.nnlib as tnn

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

In [3]:
def _get_hidden_neuron_number(i, o):
    return (max(i,o)*(min(i,o)**2))**(1/3)


class Shortcut_Conv(nn.Module):

    def __init__(self, tree, input_dim, output_dim, kernel=(3,3), stride=1):
        super().__init__()
        self.tree = tree
        self._kernel = np.array(kernel, dtype=int)
        self._padding = tuple(((self._kernel-1)/2).astype(int))
        self._stride = stride
        _wd = nn.Conv2d(input_dim, output_dim, self._kernel, stride=self._stride,
                        padding=self._padding, bias=False).weight.data
        ## Shape = OutputDim, InputDim, Kernel0, Kernel1
        self.weight = nn.Parameter(
            torch.empty_like(_wd).copy_(_wd)
        )
        del _wd
    
        ## for removing and freezing neurons
        self.to_remove = None
        self.to_freeze = None
        self.initial_remove = None
        self.initial_freeze = None
        
    def forward(self, x):
        if x.shape[1] > 0 and self.weight.shape[0] > 0:
            return F.conv2d(x, self.weight, stride=self._stride, padding=self._padding)
        ### output dim is 0
        elif self.weight.shape[0] == 0:
            ###             #num_inp  #inp_dim    #feature
            x = torch.zeros(x.shape[0], 1, x.shape[2], x.shape[3], dtype=x.dtype, device=x.device)
            ###       #out_dim #inp_dim            #kernel
            w = torch.zeros(1, 1, self.weight.shape[2], self.weight.shape[3], dtype=x.dtype, device=x.device)
            o = F.conv2d(x, w, stride=self._stride, padding=self._padding)
            return torch.zeros(o.shape[0], 0, o.shape[2], o.shape[3], dtype=x.dtype, device=x.device)
        ### input dim is 0
        elif x.shape[1] == 0:
            ###             #num_inp  #inp_dim    #feature
            x = torch.zeros(x.shape[0], 1, x.shape[2], x.shape[3], dtype=x.dtype, device=x.device)
            ###             #out_dim            #inp_dim            #kernel
            w = torch.zeros(self.weight.shape[0], 1, self.weight.shape[2], self.weight.shape[3], dtype=x.dtype, device=x.device)
            o = F.conv2d(x, w, stride=self._stride, padding=self._padding)
            return o.data
        else:
            raise(f"Unknown shape of input {x.shape} or weight {self.weight.shape}")

#     def decay_std_ratio(self, factor):
#         self.weight.data = self.weight.data - self.tree.decay_rate_std*factor.t()*self.weight.data
        
#     def decay_std_ratio_grad(self, factor):
#         self.weight.grad = self.weight.grad + self.tree.decay_rate_std*factor.t()*self.weight.data
    
    def start_decaying_connection(self, to_remove):
        self.initial_remove = self.weight.data[:, to_remove]
        self.to_remove = to_remove
        self.tree.decay_connection_shortcut.add(self)
        pass
    
    def start_freezing_connection(self, to_freeze):
        self.initial_freeze = self.weight.data[to_freeze, :]
        self.to_freeze = to_freeze
        self.tree.freeze_connection_shortcut.add(self)
        pass
    
    def freeze_connection_step(self):#, to_freeze):
        self.weight.data[self.to_freeze, :] = self.initial_freeze
        pass
    
    def decay_connection_step(self):#, to_remove):
        self.weight.data[:, self.to_remove] = self.initial_remove*self.tree.decay_factor
        pass
     
    def remove_freezed_connection(self, remaining):
        # print(self.weight.data.shape, "removing freezed; ", self.to_freeze)
        _w = self.weight.data[remaining, :]
        del self.weight
        self.weight = nn.Parameter(_w)
        self.initial_freeze = None
        self.to_freeze = None
        pass
    
    def remove_decayed_connection(self, remaining):
        # print(self.weight.data.shape, "removing decayed; ", self.to_remove)
        _w = self.weight.data[:, remaining]
        del self.weight
        self.weight = nn.Parameter(_w)
        self.initial_remove = None
        self.to_remove = None
        pass
    
    def add_input_connection(self, num):
        # print(self.weight.data.shape)
        o, i, k0, k1 = self.weight.data.shape
        _w = torch.cat((self.weight.data, torch.zeros(o, num, k0, k1, dtype=self.weight.data.dtype,
                                                      device=self.weight.data.device)), dim=1)
        del self.weight
        self.weight = nn.Parameter(_w)
        # print(self.weight.data.shape)
        pass

    def add_output_connection(self, num):
        # print(self.weight.data.shape)
        o, i, k0, k1 = self.weight.data.shape
        stdv = 1. / np.sqrt(i)
        _new = torch.empty(num, i, k0, k1, dtype=self.weight.data.dtype,
                           device=self.weight.data.device).uniform_(-stdv, stdv)
        
        _w = torch.cat((self.weight.data, _new), dim=0)
        del self.weight
        self.weight = nn.Parameter(_w)
        # print(self.weight.data.shape)        
        pass
    
    def print_network_debug(self, depth):
        print(f"{'║     '*depth}S▚:{depth}[{self.weight.data.shape[1]},{self.weight.data.shape[0]}]")



In [4]:
# class NonLinearity_Conv(nn.Module):

#     def __init__(self, tree, io_dim, actf_obj=nn.ReLU()):
#         super().__init__()
#         self.tree = tree
#         self.bias = nn.Parameter(torch.zeros(io_dim))
#         self.actf = actf_obj

#     def forward(self, x):
#         return self.actf(x+self.bias.view(1,-1,1,1))

#     def add_neuron(self, num):
#         _b = torch.cat((self.bias.data, torch.zeros(num, dtype=self.bias.data.dtype,
#                                                     device=self.bias.data.device)))
#         del self.bias
#         self.bias = nn.Parameter(_b)
        
#     def remove_neuron(self, remaining):
#         _b = self.bias.data[remaining]
#         del self.bias
#         self.bias = nn.Parameter(_b)

class NonLinearity_Conv(nn.Module):

    def __init__(self, tree, io_dim, actf_obj=nn.ReLU()):
        super().__init__()
        self.tree = tree
        self.bn = nn.BatchNorm2d(io_dim)
        self.actf = actf_obj

    def forward(self, x):
        ## if empty tensor forward that to next layer.
        if x.shape[1] < 1:
            return x
        return self.actf(self.bn(x))

    def add_neuron(self, num):
        ####https://pytorch.org/docs/stable/_modules/torch/nn/modules/batchnorm.html#BatchNorm2d
        ## running_mean
        _rm = self.bn.running_mean
        _rm = torch.cat((_rm, torch.zeros(num, dtype=_rm.dtype, device=_rm.device)))
        self.bn.running_mean = _rm
        
        ## running_var
        _rv = self.bn.running_var
        _rv = torch.cat((_rv, torch.ones(num, dtype=_rv.dtype, device=_rv.device)))
        self.bn.running_var = _rv
        
        ## weight
        _w = self.bn.weight.data
        _w = torch.cat((_w, torch.ones(num, dtype=_w.dtype, device=_w.device)*0.01))
        del self.bn.weight
        self.bn.weight = nn.Parameter(_w)
        
        ## bias
        _b = self.bn.bias.data
        _b = torch.cat((_b, torch.zeros(num, dtype=_b.dtype, device=_b.device)))
        del self.bn.bias
        self.bn.bias = nn.Parameter(_b)
        
        self.bn.num_features += num
        return
        
    def remove_neuron(self, remaining):
        ## running_mean
        _rm = self.bn.running_mean[remaining]
        self.bn.running_mean = _rm
        
        ## running_var
        _rv = self.bn.running_var[remaining]
        self.bn.running_var = _rv
        
        ## weight
        _w = self.bn.weight.data[remaining]
        del self.bn.weight
        self.bn.weight = nn.Parameter(_w)
        
        ## bias
        _b = self.bn.bias.data[remaining]
        del self.bn.bias
        self.bn.bias = nn.Parameter(_b)
        
        self.bn.num_features = len(remaining)
        return

In [5]:
class NonLinearity(nn.Module):

    def __init__(self, tree, io_dim, actf_obj=nn.ReLU()):
        super().__init__()
        self.tree = tree
        self.bias = nn.Parameter(torch.zeros(io_dim))
        self.actf = actf_obj

    def forward(self, x):
        return self.actf(x+self.bias)

    def add_neuron(self, num):
        _b = torch.cat((self.bias.data, torch.zeros(num, dtype=self.bias.data.dtype,
                                                    device=self.bias.data.device)))
        del self.bias
        self.bias = nn.Parameter(_b)
        
    def remove_neuron(self, remaining):
        _b = self.bias.data[remaining]
        del self.bias
        self.bias = nn.Parameter(_b)

In [6]:
class Residual_Conv(nn.Module):

    def __init__(self, tree, input_dim, hidden_dim, output_dim, stride=1, activation=nn.ReLU()):
        super().__init__()
        self.tree = tree
        self.hidden_dim = hidden_dim
#         self.stride = stride
        self.del_neurons = 0.
        self.neurons_added = 0

        ## Shortcut or Hierarchical Residual Layer
        self.fc0 = HierarchicalResidual_Conv(self.tree, input_dim, hidden_dim, stride=stride, activation=activation) 
        self.non_linearity = NonLinearity_Conv(self.tree, hidden_dim, activation)
        self.fc1 = HierarchicalResidual_Conv(self.tree, hidden_dim, output_dim, activation=activation)
        self.fc1.shortcut.weight.data *= 0.
        
        self.tree.parent_dict[self.fc0] = self
        self.tree.parent_dict[self.fc1] = self
        self.tree.parent_dict[self.non_linearity] = self
        
        self.hook = None
        self.activations = None
        self.significance = None
        self.count = None
        self.apnz = None
        self.to_remove = None
    
    def forward(self, x):
        x = self.fc0(x)
        x = self.non_linearity(x)
        self.activations = x.data
        x = self.fc1(x)
        return x
    
    def start_computing_significance(self):
        self.significance = 0.
        self.count = 0
        self.apnz = 0
        self.hook = self.non_linearity.register_backward_hook(self.compute_neuron_significance)
        pass
            
    def finish_computing_significance(self):
        self.hook.remove()
        self.significance = self.significance#/self.count
#         print(f"Significance before rethinking(apnz)\n{self.significance}")
#         print(f"Apnz\n{self.apnz}")
        if isinstance(self.non_linearity.actf, nn.ReLU):
            self.apnz = self.apnz/self.count
            self.significance = self.significance*(1-self.apnz) * 4 ## tried on desmos.
#         print(f"Significance after rethinking(apnz)\n{self.significance}")
#         self.count = None

        self.hook = None
        pass
    
    def compute_neuron_significance(self, _class, grad_input, grad_output):
        with torch.no_grad():
            z = torch.sum(grad_output[0].data*self.activations, dim=(2,3))
#             self.significance += z.pow(2).sum(dim=0)
            self.significance += z.abs().sum(dim=0)
#             self.significance += z.abs().pow(0.8).sum(dim=0)
#             print(f"SIG ACT:\n{float(self.activations.abs().mean())}")
#             print(f"GRAD Mean, Std:\n{float(grad_output[0].data.abs().mean()), float(grad_output[0].data.std())}")

            if isinstance(self.non_linearity.actf, nn.ReLU):
                self.count += grad_output[0].shape[0]*grad_output[0].shape[2]*grad_output[0].shape[3]
        #         self.apnz += torch.count_nonzero(self.activations.data, dim=0)
                self.apnz += torch.sum(self.activations > 0., dim=(0,2,3), dtype=z.dtype).to(z.device)
        pass
    
    def identify_removable_neurons(self, below=None, above=None, mask=None):
        if self.to_remove is not None:
            print("First remove all previous less significant neurons")
            return
        if mask is None:
            mask = torch.zeros(self.significance.numel(), dtype=torch.bool)
        if below:
            mask = torch.logical_or(mask,self.significance<=below)
        if above:
            mask = torch.logical_or(mask,self.significance>above)
            
        print(f"Significance:\n{self.significance}\nPrune:\n{mask}")
            
        self.to_remove = torch.nonzero(mask).reshape(-1)
        if len(self.to_remove)>0:
            self.fc0.start_freezing_connection(self.to_remove)
            self.fc1.start_decaying_connection(self.to_remove)
            self.tree.remove_neuron_residual.add(self)
            return len(self.to_remove)
        
        self.to_remove = None
        return 0

    def remove_decayed_neurons(self):
        remaining = []
        for i in range(self.hidden_dim):
            if i not in self.to_remove:
                remaining.append(i)
        
        self.non_linearity.remove_neuron(remaining)
        self.fc0.remove_freezed_connection(remaining)
        self.fc1.remove_decayed_connection(remaining)
        
        self.neurons_added -= len(self.to_remove)
        self.hidden_dim = len(remaining)
        self.to_remove = None
        pass
    
    def compute_del_neurons(self):
        self.del_neurons = (1-self.tree.beta_del_neuron)*self.neurons_added \
                            + self.tree.beta_del_neuron*self.del_neurons
        self.neurons_added = 0
        return
    
    def add_hidden_neuron(self, num):
        self.fc0.add_output_connection(num)
        self.non_linearity.add_neuron(num)
        self.fc1.add_input_connection(num)
        
        self.hidden_dim += num
        self.neurons_added += num
        pass

    def morph_network(self):
        self.fc0.morph_network()
        self.fc1.morph_network()
#         max_dim = np.ceil((self.tree.parent_dict[self].input_dim+\
#             self.tree.parent_dict[self].output_dim)/2)
        max_dim = _get_hidden_neuron_number(self.tree.parent_dict[self].input_dim,
            self.tree.parent_dict[self].output_dim)+1
        if self.hidden_dim <= max_dim:
            if self.fc0.residual is None:
                if self.fc0 in self.tree.DYNAMIC_LIST:
                    self.tree.DYNAMIC_LIST.remove(self.fc0)
            if self.fc1.residual is None:
                if self.fc1 in self.tree.DYNAMIC_LIST:
                    self.tree.DYNAMIC_LIST.remove(self.fc1)
        return 

    def print_network_debug(self, depth):
        print(f"{'║     '*depth}R▚:{depth}[{self.hidden_dim}|{self.non_linearity.bias.data.shape[0]}]")
        self.fc0.print_network_debug(depth+1)
        self.fc1.print_network_debug(depth+1)
        
    def print_network(self, pre_string):
        self.fc0.print_network(pre_string)
        print(f"{pre_string}{self.hidden_dim}")
        self.fc1.print_network(pre_string)
        return

In [7]:
a = torch.randn(10, 3)*0.
a.std()

tensor(0.)

In [8]:
class HierarchicalResidual_Conv(nn.Module):

    def __init__(self, tree, input_dim, output_dim, stride=1, activation=nn.ReLU()):
        super().__init__()

        self.tree = tree
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.stride = 1
        
        self.activation = activation
        
        ## this can be Shortcut Layer or None
        self.shortcut = Shortcut_Conv(tree, self.input_dim, self.output_dim, stride=self.stride).to(self.tree.device)
        self.tree.parent_dict[self.shortcut] = self
        
        self.residual = None ## this can be Residual Layer or None
        ##### only one of shortcut or residual can be None at a time
        self.forward = self.forward_shortcut
        
        self.std_ratio = 0. ## 0-> all variation due to shortcut, 1-> residual
        self.target_std_ratio = 0. ##
    
    def forward_both(self, r):
        s = self.shortcut(r)
        r = self.residual(r)

        if self.residual.hook is None: ### dont execute when computing significance
            s_std = torch.std(s, dim=(0,2,3), keepdim=True).reshape(1, -1)
            r_std = torch.std(r, dim=(0,2,3), keepdim=True).reshape(1, -1)
            stdr = r_std/(s_std+r_std)

            self.std_ratio = self.tree.beta_std_ratio*self.std_ratio + (1-self.tree.beta_std_ratio)*stdr.data
            if r_std.min() > 1e-9:
                ## recover for the fact that when decaying neurons, target ratio should also be reducing
                if self.tree.total_decay_steps:
                    i, o = self.shortcut.weight.shape[1],self.shortcut.weight.shape[0]
                    if self.shortcut.to_remove is not None:
                        i -= len(self.shortcut.to_remove)
                    if self.shortcut.to_freeze is not None:
                        o -= len(self.shortcut.to_freeze)
                    h = self.residual.hidden_dim
                    if self.residual.to_remove is not None:
                        h -= len(self.residual.to_remove)
                    
#                     tr = h/np.ceil((i+o)/2 +1)
                    tr = h/_get_hidden_neuron_number(i, o)
                    self.compute_target_std_ratio(tr)
                else:
                    self.compute_target_std_ratio()
                self.get_std_loss(stdr)
        return s+r
    
    def forward_shortcut(self, x):
        return self.shortcut(x)
    
    def forward_residual(self, x):
        self.compute_target_std_ratio()
        return self.residual(x)
    
    def compute_target_std_ratio(self, tr = None):
        if tr is None:
#             tr = self.residual.hidden_dim/np.ceil((self.input_dim+self.output_dim)/2 +1)
            tr = self.residual.hidden_dim/_get_hidden_neuron_number(self.input_dim, self.output_dim)
#             tr = self.residual.hidden_dim/np.ceil(self.output_dim/2 +1)

        tr = np.clip(tr, 0., 1.)
        self.target_std_ratio = self.tree.beta_std_ratio*self.target_std_ratio +\
                                (1-self.tree.beta_std_ratio)*tr
        pass        
    
    def get_std_loss(self, stdr):
        del_std = self.target_std_ratio-stdr
        del_std_loss = (del_std**2 + torch.abs(del_std)).mean()
#         del_std_loss = (del_std**2).mean()
        self.tree.std_loss += del_std_loss
        return
            
    def start_freezing_connection(self, to_freeze):
        if self.shortcut:
            self.shortcut.start_freezing_connection(to_freeze)
        if self.residual:
            self.residual.fc1.start_freezing_connection(to_freeze)
        pass
        
    def start_decaying_connection(self, to_remove):
        if self.shortcut:
            self.shortcut.start_decaying_connection(to_remove)
        if self.residual:
            self.residual.fc0.start_decaying_connection(to_remove)
        pass
    
    def remove_freezed_connection(self, remaining):
        if self.shortcut:
            self.shortcut.remove_freezed_connection(remaining)
        if self.residual:
            self.residual.fc1.remove_freezed_connection(remaining)
            if self.shortcut: self.std_ratio = self.std_ratio[:, remaining]
        self.output_dim = len(remaining)
        pass
    
    def remove_decayed_connection(self, remaining):
        if self.shortcut:
            self.shortcut.remove_decayed_connection(remaining)
        if self.residual:
            self.residual.fc0.remove_decayed_connection(remaining)
        self.input_dim = len(remaining)
        pass
    
    def add_input_connection(self, num):
        self.input_dim += num
        if self.shortcut:
            self.shortcut.add_input_connection(num)
        if self.residual:
            self.residual.fc0.add_input_connection(num)

    def add_output_connection(self, num):
        self.output_dim += num
        if self.shortcut:
            self.shortcut.add_output_connection(num)
        if self.residual:
            self.residual.fc1.add_output_connection(num)
            # if torch.is_tensor(self.std_ratio):
            if self.shortcut:
                self.std_ratio = torch.cat((self.std_ratio, torch.zeros(1, num, device=self.tree.device)), dim=1)

    def add_hidden_neuron(self, num):
        if num<1: return
        
        if self.residual is None:
            # print(f"Adding {num} hidden units.. in new residual_layer")
            self.residual = Residual_Conv(self.tree, self.input_dim,
                                          num, self.output_dim, stride=self.stride,
                                          activation=self.activation).to(self.tree.device)
            
            self.tree.parent_dict[self.residual] = self
            if self.shortcut is None:
                self.forward = self.forward_residual
                self.std_ratio = 1.
            else:
                self.forward = self.forward_both
                self.std_ratio = torch.zeros(1, self.output_dim, device=self.tree.device)
                
        else:
            # print(f"Adding {num} hidden units..")
            self.residual.add_hidden_neuron(num)
        return
    
    def maintain_shortcut_connection(self):
        if self.residual is None: return
        
        if self.shortcut:
            if self.std_ratio.min()>0.98 and self.target_std_ratio>0.98:
                del self.tree.parent_dict[self.shortcut]
                del self.shortcut
                self.shortcut = None
                self.forward = self.forward_residual
                self.std_ratio = 1.
            
        elif self.target_std_ratio<0.95:
            self.shortcut = Shortcut_Conv(self.tree, self.input_dim, self.output_dim, stride=self.stride)
            self.shortcut.weight.data *= 0.
            self.forward = self.forward_both
            
        self.residual.fc0.maintain_shortcut_connection()
        self.residual.fc1.maintain_shortcut_connection()
        
    def morph_network(self):
        if self.residual is None: return
        
        if self.residual.hidden_dim < 1:
            del self.tree.parent_dict[self.residual]
            del self.residual
            ### its parent (Residual_Conv) removes it from dynamic list if possible
            self.residual = None
            self.forward = self.forward_shortcut
            self.std_ratio = 0.
            return
        
#         max_dim = np.ceil((self.input_dim+self.output_dim)/2)
        # max_dim = min((self.input_dim, self.output_dim))+1
        max_dim = _get_hidden_neuron_number(self.input_dim, self.output_dim) + 1 
        # print("MaxDIM", max_dim, self.residual.hidden_dim)
        if self.residual.hidden_dim > max_dim:
            self.tree.DYNAMIC_LIST.add(self.residual.fc0)
            self.tree.DYNAMIC_LIST.add(self.residual.fc1)
            # print("Added", self.residual)
            
        # self.residual.fc0.morph_network()
        # self.residual.fc1.morph_network()
        self.residual.morph_network()
        
    def print_network_debug(self, depth):
        stdr = self.std_ratio
        if torch.is_tensor(self.std_ratio):
            stdr = self.std_ratio.min()
            
        print(f"{'|     '*depth}H:{depth}[{self.input_dim},{self.output_dim}]"+\
              f"σ[t:{self.target_std_ratio}, s:{stdr}")
        if self.shortcut:
            self.shortcut.print_network_debug(depth+1)
        if self.residual:
            self.residual.print_network_debug(depth+1)
        pass
    
    def print_network(self, pre_string=""):
        if self.residual is None:
            return
        
        if self.shortcut:
            print(f"{pre_string}╠════╗")
            self.residual.print_network(f"{pre_string}║    ")
            print(f"{pre_string}╠════╝")
        else:
            print(f"{pre_string}╚════╗")
            self.residual.print_network(f"{pre_string}     ")
            print(f"{pre_string}╔════╝")
        return

### Conv Conv Connector

In [9]:
class Residual_Conv_Connector(nn.Module):

    def __init__(self, tree, hrnet0, hrnet1, activation, hidden_dim, post_activation=None):
        super().__init__()
        self.tree = tree
        self.hidden_dim = hidden_dim
        self.del_neurons = 0.
        self.neurons_added = 0
        self.post_activation = post_activation

        ## Shortcut or Hierarchical Residual Layer
        self.fc0 = hrnet0
        self.non_linearity = NonLinearity_Conv(self.tree, hidden_dim, activation)
        self.fc1 = hrnet1
        
        self.tree.parent_dict[self.fc0] = self
        self.tree.parent_dict[self.fc1] = self
        self.tree.parent_dict[self.non_linearity] = self
        
        self.hook = None
        self.activations = None
        self.significance = None
        self.count = None
        self.apnz = None
        self.to_remove = None
    
    def forward(self, x):
        x = self.fc0(x)
        x = self.non_linearity(x)
        self.activations = x.data
        if self.post_activation:
            x = self.post_activation(x)
        x = self.fc1(x)
        return x
    
    def start_computing_significance(self):
        self.significance = 0.
        self.count = 0
        self.apnz = 0
        self.hook = self.non_linearity.register_backward_hook(self.compute_neuron_significance)
        pass
            
    def finish_computing_significance(self):
        self.hook.remove()
        self.significance = self.significance#/self.count
#         print(f"Significance before rethinking(apnz)\n{self.significance}")
#         print(f"Apnz\n{self.apnz}")
        if isinstance(self.non_linearity.actf, nn.ReLU):
            self.apnz = self.apnz/self.count
            self.significance = self.significance*(1-self.apnz) * 4 ## tried on desmos.
#         print(f"Significance after rethinking(apnz)\n{self.significance}")
#         self.count = None

        self.hook = None
        pass
    
    
    def compute_neuron_significance(self, _class, grad_input, grad_output):
        with torch.no_grad():
            z = torch.sum(grad_output[0].data*self.activations, dim=(2,3))
#             self.significance += z.pow(2).sum(dim=0)
            self.significance += z.abs().sum(dim=0)
#             self.significance += z.abs().pow(0.8).sum(dim=0)
#             print("Current Significance \n", self.significance)
#             print(f"SIG ACT:\n{float(self.activations.abs().mean())}")
#             print(f"GRAD Mean, Std:\n{float(grad_output[0].data.abs().mean()), float(grad_output[0].data.std())}")

            if isinstance(self.non_linearity.actf, nn.ReLU):
                self.count += grad_output[0].shape[0]*grad_output[0].shape[2]*grad_output[0].shape[3]
        #         self.apnz += torch.count_nonzero(self.activations.data, dim=0)
                self.apnz += torch.sum(self.activations > 0., dim=(0,2,3), dtype=z.dtype).to(z.device)
        pass
    
    def identify_removable_neurons(self, below=None, above=None, mask=None):
        if self.to_remove is not None:
            print("First remove all previous less significant neurons")
            return
        if mask is None:
            mask = torch.zeros(self.significance.numel(), dtype=torch.bool)
        if below:
            mask = torch.logical_or(mask,self.significance<=below)
        if above:
            mask = torch.logical_or(mask,self.significance>above)
            
        print(f"Significance:\n{self.significance}\nPrune:\n{mask}")
        
        self.to_remove = torch.nonzero(mask).reshape(-1)
        if len(self.to_remove)>0:
            self.fc0.start_freezing_connection(self.to_remove)
            self.fc1.start_decaying_connection(self.to_remove)
            self.tree.remove_neuron_residual.add(self)
            return len(self.to_remove)
        
        self.to_remove = None
        return 0

    def remove_decayed_neurons(self):
        remaining = []
        for i in range(self.hidden_dim):
            if i not in self.to_remove:
                remaining.append(i)
        
        self.non_linearity.remove_neuron(remaining)
        self.fc0.remove_freezed_connection(remaining)
        self.fc1.remove_decayed_connection(remaining)
        
        self.neurons_added -= len(self.to_remove)
        self.hidden_dim = len(remaining)
        self.to_remove = None
        pass
    
    def compute_del_neurons(self):
        self.del_neurons = (1-self.tree.beta_del_neuron)*self.neurons_added \
                            + self.tree.beta_del_neuron*self.del_neurons
        self.neurons_added = 0
        return
    
    def add_hidden_neuron(self, num):
        self.fc0.add_output_connection(num)
        self.non_linearity.add_neuron(num)
        self.fc1.add_input_connection(num)
        
        self.hidden_dim += num
        self.neurons_added += num
        pass

    def morph_network(self):
        self.fc0.morph_network()
        self.fc1.morph_network()
        max_dim = _get_hidden_neuron_number(self.tree.parent_dict[self].input_dim,
            self.tree.parent_dict[self].output_dim)+1
        if self.hidden_dim <= max_dim:
            if self.fc0.residual is None:
                if self.fc0 in self.tree.DYNAMIC_LIST:
                    self.tree.DYNAMIC_LIST.remove(self.fc0)
            if self.fc1.residual is None:
                if self.fc1 in self.tree.DYNAMIC_LIST:
                    self.tree.DYNAMIC_LIST.remove(self.fc1)
        return 

    def print_network_debug(self, depth):
        print(f"{'║     '*depth}R▚:{depth}[{self.hidden_dim}|{self.non_linearity.bias.data.shape[0]}]")
        self.fc0.print_network_debug(depth+1)
        self.fc1.print_network_debug(depth+1)
        
    def print_network(self, pre_string):
        self.fc0.print_network(pre_string)
        print(f"{pre_string}{self.hidden_dim}")
        self.fc1.print_network(pre_string)
        return

In [10]:
a = torch.randn(10)<0 
b = torch.randn(10) > 0.5
torch.nonzero(torch.logical_and(a,b))

<ipython-input-10-96a8be50163c>:3: UserWarning: This overload of nonzero is deprecated:
	nonzero(Tensor input, *, Tensor out)
Consider using one of the following signatures instead:
	nonzero(Tensor input, *, bool as_tuple) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:766.)
  torch.nonzero(torch.logical_and(a,b))


tensor([[3]])

In [11]:
class HierarchicalResidual_Connector(nn.Module):

    def __init__(self, tree, hrnet0, hrnet1, activation=nn.ReLU(), post_activation=None):
        super().__init__()

        self.tree = tree
        self.input_dim = hrnet0.input_dim
        self.output_dim = hrnet1.output_dim
        
        ## this can be Shortcut Layer or None
        self.shortcut = None
        self.residual = Residual_Conv_Connector(self.tree, hrnet0, hrnet1,
                                                activation, hrnet0.output_dim, post_activation)
        self.tree.parent_dict[self.residual] = self
            
    
    def forward(self, x):
        return self.residual(x)
    
    def start_freezing_connection(self, to_freeze):
        self.residual.fc1.start_freezing_connection(to_freeze)
        pass
        
    def start_decaying_connection(self, to_remove):
        self.residual.fc0.start_decaying_connection(to_remove)
        pass
    
    def remove_freezed_connection(self, remaining):
        self.residual.fc1.remove_freezed_connection(remaining)
        self.output_dim = len(remaining)
        pass
    
    def remove_decayed_connection(self, remaining):
        self.residual.fc0.remove_decayed_connection(remaining)
        self.input_dim = len(remaining)
        pass
    
    def add_input_connection(self, num):
        self.input_dim += num
        self.residual.fc0.add_input_connection(num)

    def add_output_connection(self, num):
        self.output_dim += num
        self.residual.fc1.add_output_connection(num)
        
    def add_hidden_neuron(self, num):
        if num<1: return
        self.residual.add_hidden_neuron(num)
        return
    
    def maintain_shortcut_connection(self):  
        self.residual.fc0.maintain_shortcut_connection()
        self.residual.fc1.maintain_shortcut_connection()
        
    def morph_network(self):
        self.residual.morph_network()
        
    def print_network_debug(self, depth):
        print(f"{'|     '*depth}H:{depth}[{self.input_dim},{self.output_dim}]"+\
              f"σ[t:{None}, s:{None}")
        self.residual.print_network_debug(depth+1)
        pass
    
    def print_network(self, pre_string=""):
        print(f"{pre_string}╚╗")
        self.residual.print_network(f"{pre_string} ")
        print(f"{pre_string}╔╝")
        return

## Shortcut only Hierarchical Residual Network

In [12]:
class Shortcut(nn.Module):

    def __init__(self, tree, input_dim, output_dim):
        super().__init__()
        self.tree = tree
        _wd = nn.Linear(input_dim, output_dim, bias=False).weight.data
        self.weight = nn.Parameter(
            torch.empty_like(_wd).copy_(_wd)
        )
    
        ## for removing and freezing neurons
        self.to_remove = None
        self.to_freeze = None
        self.initial_remove = None
        self.initial_freeze = None
    
    def forward(self, x):
        ## input_dim        ## output_dim
        if x.shape[1] + self.weight.shape[1] > 0:
            return x.matmul(self.weight.t())
        else:
            # print(x.shape, self.weight.shape)
            # print(x.matmul(self.weight.t()))
            if x.shape[1] + self.weight.shape[1] == 0:
                return torch.zeros(x.shape[0], self.weight.shape[0], dtype=x.dtype, device=x.device)
        
    def start_decaying_connection(self, to_remove):
        self.initial_remove = self.weight.data[:, to_remove]
        self.to_remove = to_remove
        self.tree.decay_connection_shortcut.add(self)
        pass
    
    def start_freezing_connection(self, to_freeze):
        self.initial_freeze = self.weight.data[to_freeze, :]
        self.to_freeze = to_freeze
        self.tree.freeze_connection_shortcut.add(self)
        pass
    
    def freeze_connection_step(self):#, to_freeze):
        self.weight.data[self.to_freeze, :] = self.initial_freeze
        pass
    
    def decay_connection_step(self):#, to_remove):
        self.weight.data[:, self.to_remove] = self.initial_remove*self.tree.decay_factor
        pass
            
     
    def remove_freezed_connection(self, remaining):
        # print(self.weight.data.shape, "removing freezed; ", self.to_freeze)
        _w = self.weight.data[remaining, :]
        del self.weight
        self.weight = nn.Parameter(_w)
        self.initial_freeze = None
        self.to_freeze = None
        pass
    
    def remove_decayed_connection(self, remaining):
        # print(self.weight.data.shape, "removing decayed; ", self.to_remove)
        _w = self.weight.data[:, remaining]
        del self.weight
        self.weight = nn.Parameter(_w)
        self.initial_remove = None
        self.to_remove = None
        pass
    
    def add_input_connection(self, num):
        # print(self.weight.data.shape)
        o, i = self.weight.data.shape
        _w = torch.zeros(o, num, dtype=self.weight.data.dtype, device=self.weight.data.device)
        _w += torch.randn_like(_w)*0.01
        _w = torch.cat((self.weight.data, _w), dim=1)
        del self.weight
        self.weight = nn.Parameter(_w)
        # print(self.weight.data.shape)
        pass

    def add_output_connection(self, num):
        # print(self.weight.data.shape)
        o, i = self.weight.data.shape
        stdv = 1. / np.sqrt(i)
        _new = torch.empty(num, i, dtype=self.bias.weight.dtype,
                           device=self.weight.data.device).uniform_(-stdv, stdv)
        
        _w = torch.cat((self.weight.data, _new), dim=0)
        del self.weight
        self.weight = nn.Parameter(_w)
        # print(self.weight.data.shape)        
        pass
    
    def print_network_debug(self, depth):
        print(f"{'|     '*depth}S:{depth}[{self.weight.data.shape[1]},{self.weight.data.shape[0]}]")


class HierarchicalResidual_Shortcut(nn.Module):

    def __init__(self, tree, input_dim, output_dim, kernel=None, stride=1):
        super().__init__()

        self.tree = tree
        self.input_dim = input_dim
        self.output_dim = output_dim
        ## this can be Shortcut Layer or None
        if kernel is None:
            self.shortcut = Shortcut(tree, self.input_dim, self.output_dim) 
        else:
            self.shortcut = Shortcut_Conv(tree, self.input_dim, self.output_dim, kernel, stride) 
        self.tree.parent_dict[self.shortcut] = self
        
        self.residual = None
    
    def forward(self, x):
        return self.shortcut(x)
    
    def start_freezing_connection(self, to_freeze):
        self.shortcut.start_freezing_connection(to_freeze)
        
    def start_decaying_connection(self, to_remove):
        self.shortcut.start_decaying_connection(to_remove)
    
    def remove_freezed_connection(self, remaining):
        self.shortcut.remove_freezed_connection(remaining)
        self.output_dim = len(remaining)
    
    def remove_decayed_connection(self, remaining):
        self.shortcut.remove_decayed_connection(remaining)
        self.input_dim = len(remaining)
        pass
    
    def add_input_connection(self, num):
        self.input_dim += num
        self.shortcut.add_input_connection(num)

    def add_output_connection(self, num):
        self.output_dim += num
        self.shortcut.add_output_connection(num)

    def add_hidden_neuron(self, num):
        print("Cannot Add Hidden neuron to Shortcut Only Layer")
        return
    
    def maintain_shortcut_connection(self):
        pass
        
    def morph_network(self):
        pass
        
    def print_network_debug(self, depth):
        print(f"{'|     '*depth}H:{depth}[{self.input_dim},{self.output_dim}]"+\
              f"σ[t:{None}, s:{None}")
        self.shortcut.print_network_debug(depth+1)
        pass
    
    def print_network(self, pre_string=""):
        pass

# Tree and Controller

In [13]:
class Tree_State():
    
    def __init__(self):
        self.DYNAMIC_LIST = set() ## residual parent is added, to make code effecient.
        ## the parents which is not intended to have residual connection should not be added.
        self.beta_std_ratio = None
        self.beta_del_neuron = None
        self.device = 'cpu'
    
        self.parent_dict = {}
    
        self.total_decay_steps = None
        self.current_decay_step = None
        self.decay_factor = None
        self.remove_neuron_residual:set = None
        self.freeze_connection_shortcut:set = None
        self.decay_connection_shortcut:set = None

        self.decay_rate_std = 0.001

        self.add_to_remove_ratio = 2.
        pass
    
    def get_decay_factor(self):
        ratio = self.current_decay_step/self.total_decay_steps
#         self.decay_factor = np.exp(-2*ratio)*(1-ratio)
        self.decay_factor = (1-ratio)**2
        pass
    
    def clear_decay_variables(self):
        self.total_decay_steps = None
        self.current_decay_step = None
        self.decay_factor = None
        self.remove_neuron_residual = None
        self.freeze_connection_shortcut = None
        self.decay_connection_shortcut = None
        

## constructing Hierarchical Residual CNN (Resnet Inspired)

In [14]:
class DropoutActivation(nn.Module):
    
    def __init__(self, p=0.3, activation=nn.ReLU()):
        super().__init__()
        self.dropout = nn.Dropout2d(p)
        self.activation = activation
        
    def forward(self, x):
        return self.activation(self.dropout(x))

In [15]:
class Dynamic_CNN(nn.Module):

    def __init__(self, device, input_dim = 1, hidden_dims = [8, 16, 32, 64], output_dim = 10, final_activation=None,
                 num_stat=5, num_std=100, decay_rate_std=0.001):
        super().__init__()
        self.tree = Tree_State()
        self.tree.beta_del_neuron = (num_stat-1)/num_stat
        self.tree.beta_std_ratio = (num_std-1)/num_std
        self.tree.decay_rate_std = decay_rate_std
        self.tree.device = device
        
        self.root_net = None
        self._construct_root_net(input_dim, hidden_dims, output_dim)
        
#         self.tree.DYNAMIC_LIST.add(self.root_net)
        self.tree.parent_dict[self.root_net] = None
        
        if final_activation is None:
            final_activation = lambda x: x
        self.non_linearity = NonLinearity("Root", output_dim, final_activation)
        
        self.neurons_added = 0

        self._remove_below = None ## temporary variable
        
    def _construct_root_net(self, input_dim, hidden_dims, output_dim):
        
        actf = DropoutActivation()
#         actf = lambda x: x
#         actf = nn.ReLU()

        hrnR = HierarchicalResidual_Shortcut(self.tree, 3, 8, kernel=(3,3), stride=1)
        hrn0 = HierarchicalResidual_Conv(self.tree, 8, 8, activation=actf)
        hrn1 = HierarchicalResidual_Conv(self.tree, 8, 16, stride=2, activation=actf)
        hrn2 = HierarchicalResidual_Conv(self.tree, 16, 32, stride=2, activation=actf)
        hrn3 = HierarchicalResidual_Conv(self.tree, 32, 32, stride=2, activation=actf)

    
        actf = lambda x: x
        hrnR0 = HierarchicalResidual_Connector(self.tree, hrnR, hrn0, actf)
        hrnR01 = HierarchicalResidual_Connector(self.tree, hrnR0, hrn1, actf)
        hrnR012 = HierarchicalResidual_Connector(self.tree, hrnR01, hrn2, actf)
        hrnR0123 = HierarchicalResidual_Connector(self.tree, hrnR012, hrn3, actf)
        hrnfc = HierarchicalResidual_Shortcut(self.tree, 32, 10)
        
        def pool_and_reshape(x):
            x = F.adaptive_avg_pool2d(x, (1,1))
            x = x.view(x.shape[0], -1)
            return x
        
#         actf = lambda x: x
#         actf = nn.ReLU()

        hrnR0123fc = HierarchicalResidual_Connector(self.tree, hrnR0123, hrnfc,
                                                   activation=actf, post_activation=pool_and_reshape)
        self.root_net = hrnR0123fc
        
        ## make every hierarchical Layer Morphable
        morphables = [self.root_net, hrnR0123, hrnR012, hrnR01, hrnR0, hrn3, hrn2, hrn1, hrn0]
#         morphables = [self.root_net, hrn0123, hrn012, hrn01]
        for hr in morphables:
            self.tree.DYNAMIC_LIST.add(hr)
        return
    
    def _construct_root_net2(self, input_dim, hidden_dims, output_dim):
        
        
        
        hrnR = HierarchicalResidual_Shortcut(self.tree, 3, 16, kernel=(3,3), stride=1)
        hrn0 = HierarchicalResidual_Conv(self.tree, 16, 16)
        hrn1 = HierarchicalResidual_Conv(self.tree, 16, 32, stride=2)
        hrn2 = HierarchicalResidual_Conv(self.tree, 32, 64, stride=2)

        actf = lambda x: x
#         actf = nn.ReLU()
    
        hrnR0 = HierarchicalResidual_Connector(self.tree, hrnR, hrn0)
        hrnR01 = HierarchicalResidual_Connector(self.tree, hrnR0, hrn1, actf)
        hrnR012 = HierarchicalResidual_Connector(self.tree, hrnR01, hrn2, actf)
        hrnfc = HierarchicalResidual_Shortcut(self.tree, 64, 10)
        
        def pool_and_reshape(x):
            x = F.adaptive_avg_pool2d(x, (1,1))
            x = x.view(x.shape[0], -1)
            return x
        
#         actf = lambda x: x
#         actf = nn.ReLU()

        hrnR012fc = HierarchicalResidual_Connector(self.tree, hrnR012, hrnfc,
                                                   activation=actf, post_activation=pool_and_reshape)
        self.root_net = hrnR012fc
        
        ## make every hierarchical Layer Morphable
        morphables = [hrn2, hrn1, hrn0]
#         morphables = [self.root_net, hrnR012, hrnR01, hrnR0, hrn2, hrn1, hrn0]
#         morphables = [self.root_net, hrn0123, hrn012, hrn01]
        for hr in morphables:
            self.tree.DYNAMIC_LIST.add(hr)
        return
    
    def forward(self, x):
        return self.non_linearity(self.root_net(x))

    def add_neurons(self, num):
        num_stat = num//2
        num_random = num - num_stat
        
        DL = list(self.tree.DYNAMIC_LIST)
        if num_random>0:
            rands = torch.randint(high=len(DL), size=(num_random,))
            index, count = torch.unique(rands, sorted=False, return_counts=True)
            for i, idx in enumerate(index):
                DL[idx].add_hidden_neuron(int(count[i]))

        if num_stat>0:
            del_neurons = []
            for hr in DL:
                if hr.residual:
                    del_neurons.append(hr.residual.del_neurons)#+1e-7)
                else:
                    del_neurons.append(0.)#1e-7) ## residual layer yet not created 
            
            prob_stat = torch.tensor(del_neurons)
            prob_stat = torch.log(torch.exp(prob_stat)+1.)
            m = torch.distributions.multinomial.Multinomial(total_count=num_stat,
                                                            probs= prob_stat)
            count = m.sample()#.type(torch.long)
            for i, hr in enumerate(DL):
                if count[i] < 1: continue
                hr.add_hidden_neuron(int(count[i]))
        
        self.neurons_added += num 
        pass

    def identify_removable_neurons(self, num=None, threshold_min=0., threshold_max=1.):
        
        all_sig = []
        self.all_sig_ = []
        
        for hr in self.tree.DYNAMIC_LIST:
            if hr.residual:
                all_sig.append(hr.residual.significance)
                
        all_sigs = torch.cat(all_sig)
        del all_sig
        
#         print("All_sigs", all_sigs)
        
#         print("Normalization", (all_sigs/all_sigs.sum()).sum())
        
        ### Normalizes such that importance 1 is average importance
        normalizer = float(torch.sum(all_sigs))/len(all_sigs)
        all_sig = all_sigs/normalizer

        ### Normalizes to range [0, 1]
#         max_sig = all_sigs.max()
#         all_sig = all_sigs/(max_sig+1e-9)
#         print("All_sig", all_sig)
#         print("Sig sum", all_sig.sum())
        print(f"Significance Stat:\nMin, Max: {float(all_sig.min()), float(all_sig.max())}")
        print(f"Mean, Std: {float(all_sig.mean()), float(all_sig.std())}")
        all_sig = all_sig[all_sig<threshold_max]
        if len(all_sig)<1: ## if all significance is above threshold max 
            return 0, None, all_sigs
        all_sig = torch.sort(all_sig)[0] ### sorted significance scores
        
        self.all_sig_ = all_sig
        
        if not num:num = int(np.ceil(self.neurons_added/self.tree.add_to_remove_ratio))
        ## reset the neurons_added number if decay is started

        remove_below = threshold_min
        if num>len(all_sig):
            remove_below = float(all_sig[-1])
        elif num>0:
            remove_below = float(all_sig[num-1])
        
        ### sig < threshold_min is always removed; whatsoever
        if remove_below < threshold_min:
            remove_below = threshold_min
            
        print("remove_below", remove_below)
        remove_below *= normalizer
#         remove_below *= max_sig
#         print("remove_below", remove_below)

        self._remove_below = remove_below
#         self._remove_above = remove_above*normalizer
        self._remove_above = None

        return remove_below, all_sigs

    def decay_neuron_start(self, decay_steps=1000):
        if self._remove_below is None: return 0
        
        self.neurons_added = 0 ## resetting this variable
        
        self.tree.total_decay_steps = decay_steps
        self.tree.current_decay_step = 0
        self.tree.remove_neuron_residual = set()
        self.tree.freeze_connection_shortcut = set()
        self.tree.decay_connection_shortcut = set()
        
        count_remove = 0
        for hr in self.tree.DYNAMIC_LIST:
            if hr.residual:
                ### always prune 1 % of the neurons randomly. It might overlap with less significant neurons
                mask = torch.bernoulli(torch.ones_like(hr.residual.significance)*0.05).type(torch.bool)
                count_remove += hr.residual.identify_removable_neurons(below=self._remove_below,
                                                                       above=self._remove_above,
                                                                       mask = mask
                                                                      )
        if count_remove<1:
            self.tree.clear_decay_variables()
        return count_remove
    
    def decay_neuron_step(self):
        if self.tree.total_decay_steps is None:
            return
        
        self.tree.current_decay_step += 1
        
        if self.tree.current_decay_step < self.tree.total_decay_steps:
            self.tree.get_decay_factor()
            for sh in self.tree.decay_connection_shortcut:
                sh.decay_connection_step()
            for sh in self.tree.freeze_connection_shortcut:
                sh.freeze_connection_step()
        else:
            for rs in self.tree.remove_neuron_residual:
                rs.remove_decayed_neurons()
            
            self.tree.clear_decay_variables()
            
    def compute_del_neurons(self):
        for hr in self.tree.DYNAMIC_LIST:
            if hr.residual:
                hr.residual.compute_del_neurons()
    
    def maintain_network(self):
        self.root_net.maintain_shortcut_connection()
        self.root_net.morph_network()
        
    def start_computing_significance(self):
        for hr in self.tree.DYNAMIC_LIST:
            if hr.residual:
                hr.residual.start_computing_significance()

    def finish_computing_significance(self):
        for hr in self.tree.DYNAMIC_LIST:
            if hr.residual:
                hr.residual.finish_computing_significance()
            
    def print_network_debug(self):
        self.root_net.print_network_debug(0)
        
    def print_network(self):
        print(self.root_net.input_dim)
        self.root_net.print_network()
        print("│")
        print(self.root_net.output_dim)
        return

In [16]:
# np.random.binomial(1, 0.01, 100)

In [17]:
# torch.bernoulli(torch.ones(10)*0.01).type(torch.bool)

## Train dycnn

In [18]:
device = torch.device('cuda:1')

In [19]:
from torchvision import datasets, transforms

In [20]:
cifar_train = transforms.Compose([
    transforms.RandomCrop(size=32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.4914, 0.4822, 0.4465], # mean=[0.5071, 0.4865, 0.4409] for cifar100
        std=[0.2023, 0.1994, 0.2010], # std=[0.2009, 0.1984, 0.2023] for cifar100
    ),
])

cifar_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.4914, 0.4822, 0.4465], # mean=[0.5071, 0.4865, 0.4409] for cifar100
        std=[0.2023, 0.1994, 0.2010], # std=[0.2009, 0.1984, 0.2023] for cifar100
    ),
])

train_dataset = datasets.CIFAR10(root="../../_Datasets/cifar10/", train=True, download=True, transform=cifar_train)
test_dataset = datasets.CIFAR10(root="../../_Datasets/cifar10/", train=False, download=True, transform=cifar_test)

Files already downloaded and verified
Files already downloaded and verified


In [21]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=32, shuffle=True, num_workers=4)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=32, shuffle=False, num_workers=4)

In [22]:
learning_rate = 0.01
# learning_rate = 0.00003

dynet = Dynamic_CNN(device).to(device)
criterion = nn.CrossEntropyLoss()

In [23]:
len(train_loader), len(test_loader)

(1563, 313)

In [24]:
### hyperparameters
num_add_neuron = 50#25#10
num_decay_steps = int(len(train_loader)*3)#3

remove_above = 10
threshold_max = 1
threshold_min = 0.01

train_epoch_min = 1 #1
train_epoch_max = 6 #5

In [25]:
dynet.tree.add_to_remove_ratio = 1.7

In [26]:
num_decay_steps

4689

## Auto_Training

In [27]:
class AutoTrainer:
    
    def __init__(self):
        self.training_func = None
        self.adding_func = None
        self.pruning_func = None
        self.maintainance_func = None
        self.extra_func = None
        
        self.log_func = None
        
    def loop(self, count = 15):
        cb = count
        for i in range(count):
            if i>-0.1:
                self.adding_func()
            else:
                global optimizer, warmup
                dynet.print_network()    
                optimizer = torch.optim.Adam(dynet.parameters(), lr=learning_rate)
#                 optimizer = torch.optim.SGD(dynet.parameters(), lr=learning_rate, momentum=0.9)
                warmup = WarmupLR_Polynomial(optimizer, 0, len(train_loader))
            
            self.training_func()
            
            if i>-0.1:
                self.pruning_func()
            
            self.maintainance_func()
            
            self.log_func(i)
            
            if self.extra_func:
                self.extra_func()
            
            print(f"=====================")
            print(f"===LOOPS FINISHED :{i} ===")
            print(f"Pausing for 2 second to give user time to STOP PROCESS")
            time.sleep(2)
        self.training_func()

### when to stop training functionality

In [28]:
def update_coeff(num_iter, coeff0, coeff1, coeff2, coeff_opt, loss_list):
    if len(loss_list)<10: return np.array([0]), np.array([0]), float(coeff0.data.cpu()[0])
    
    _t = torch.tensor(loss_list)
    _t = (_t - _t[-1])/(_t[0]-_t.min()) ## normalize to make first point at 1 and last at 0 
    _t = torch.clamp(_t, -1.1, 1.1)
    _x = torch.linspace(0, 1, steps=len(_t))
    
    for _ in range(num_iter):
        coeff_opt.zero_grad()
        _y = torch.exp(coeff0*_x)*(1-_x)*coeff1 + coeff2

        _loss = ((_y - _t)**2).mean()
        _loss.backward()
        coeff_opt.step()

        coeff0.data = torch.clamp(coeff0.data, -20., 20.)
        coeff1.data = torch.clamp(coeff1.data, 0.7, 2.)
        coeff2.data = torch.clamp(coeff2.data, -0.2,0.1)
        
    if torch.isnan(coeff0.data[0]):
        coeff0.data[0] = 0.
        coeff1.data[0] = 0.
        coeff2.data[0] = 1. ## this gives signal
        
    _y = torch.exp(coeff0*_x)*(1-_x)*coeff1 + coeff2

    return _x.numpy(), _t.numpy(), _y.data.cpu().numpy()

## Train Network dynamically

In [29]:
## global variables
optimizer = None
warmup = None
coeff_opt = None

loss_all = []
accs_all = []
accs_test = []

## for adam optimizer = 
learning_rate *= 0.1

In [30]:
class WarmupLR_Polynomial():
    
    def __init__(self, optimizer, warmup_epoch, num_batch_in_epoch, power=5):
        self.warmup_epoch = warmup_epoch
        self.optimizer = optimizer
        self.num_batch = num_batch_in_epoch
        self.steps = 0
        self.power = power
        
    def step(self):
        steps = self.steps/self.num_batch
        self.steps += 1
        
        factor = 1
        warming = False
        if steps<self.warmup_epoch:
            factor = (steps/self.warmup_epoch)**self.power
            warming = True
            
        for group in self.optimizer.param_groups:
            group['lr'] *= factor
        
        return warming

In [31]:
def add_neurons_func():
    global optimizer, warmup, added
    ### number of neurons
    count = 0
    for hr in dynet.tree.DYNAMIC_LIST:
        if hr.residual:
            count += hr.residual.hidden_dim
    ## add more neurons relatively (+x%)
    adding = num_add_neuron+int(0.07*count)
    dynet.add_neurons(adding)
    print(f"Adding {adding} Neurons")
    added = adding
    dynet.print_network()    
    optimizer = torch.optim.Adam(dynet.parameters(), lr=learning_rate)
#     optimizer = torch.optim.SGD(dynet.parameters(), lr=learning_rate, momentum=0.9)
    warmup = WarmupLR_Polynomial(optimizer, 0, len(train_loader), power=1)
    return

In [32]:
def training_network_func():
    global optimizer, warmup, loss_all, accs_all
    
    coeff0 = torch.zeros(1, requires_grad=True)
    coeff1 = torch.zeros(1, requires_grad=True)
    coeff2 = torch.zeros(1, requires_grad=True)
    coeff_opt = torch.optim.Adam([coeff0, coeff1, coeff2], lr=0.8)
    loss_list = []
    prev_loss = None
    beta_loss = (1000-1)/1000
    loss_ = []
    
    fig = plt.figure(figsize=(10,4))
    ax = fig.add_subplot(121)
    ax2 = fig.add_subplot(122)
    breakall=False
    

    steps_ = -1
    for epoch in range(train_epoch_max):
        
        train_acc = 0
        train_count = 0
        for train_x, train_y in train_loader:
            train_x, train_y = train_x.to(device), train_y.to(device)
            steps_ += 1
            
            dynet.decay_neuron_step()
            dynet.tree.std_loss = 0.    

            yout = dynet(train_x)
            loss = criterion(yout, train_y) + dynet.tree.decay_rate_std*dynet.tree.std_loss
            
            if steps_>100:
                prev_loss = (1-beta_loss)*float(loss)+beta_loss*prev_loss
                loss_list.append(prev_loss)
            elif steps_ == 100:
                loss_.append(float(loss))
                prev_loss = np.mean(loss_)
                loss_ = []
            else:
                loss_.append(float(loss))
                    
            optimizer.zero_grad()
            loss.backward(retain_graph=False)
            warmup.step()
            optimizer.step()
            
            outputs = tnn.Logits.logit_to_index(yout.data.cpu().numpy())
            targets = train_y.data.cpu().numpy()

            correct = (outputs == targets).sum()
            train_acc += correct
            train_count += len(outputs)

            if steps_%100 == 0 and steps_>0:
                if len(loss_list)>0:
                    max_indx = np.argmax(loss_list)
                    loss_list = loss_list[max_indx:]
    #                 loss_all.append(float(loss))
                
                _x, _t, _y = update_coeff(50, coeff0, coeff1, coeff2, coeff_opt, loss_list)
                _c = float(coeff0.data.cpu()[0])
    #             if coeff2.data[0] > 0.5: ## this is a signal to reset optimizer
                coeff_opt = torch.optim.Adam([coeff0, coeff1, coeff2], lr=0.8)
                _info = f'ES: {epoch}:{steps_}, coeff:{_c:.3f}/{-5}, \nLoss:{float(loss):.3f}, Acc:{correct/len(outputs)*100:.3f}%'

                ax.clear()
                if len(_x)>0:
                    ax.plot(_x, _t, c='c')
                    ax.plot(_x, _y, c='m')
                xmin, xmax = ax.get_xlim()
                ymin, ymax = ax.get_ylim()
                ax.text(xmin, ymin, _info)
                    
                ax2.clear()
                if len(accs_all)>0:
                    acc_tr = accs_all
                    acc_te = accs_test
                    if len(acc_tr)>20: acc_tr = acc_tr[-20:]
                    if len(acc_te)>20: acc_te = acc_te[-20:]
                    ax2.plot(acc_tr, label="train")
                    ax2.plot(acc_te, label="test")
                    ax2.legend(loc="lower right")
                    
                    ymin, ymax = ax2.get_ylim()
                    ax2.text(0, 0.1*ymin+0.9*ymax, f"TR:max{max(acc_tr):.3f} end{acc_tr[-1]:.3f}")
                    ax2.text(0, 0.2*ymin+0.8*ymax, f"TE:max{max(acc_te):.3f} end{acc_te[-1]:.3f}")

                
                fig.canvas.draw()
                plt.savefig("./files/_temp_train_plot2.png")

                if _c < -5 and epoch>train_epoch_min: 
                    breakall=True
                    break
                    
        if not breakall:
            accs_all.append(train_acc/train_count*100.)
            with torch.no_grad():
                corrects = 0
                dynet.eval()
                for test_x, test_y in test_loader:
                    test_x  = test_x.to(device)
                    yout = dynet.forward(test_x)
                    outputs = tnn.Logits.logit_to_index(yout.data.cpu().numpy())
                    correct = (outputs == test_y.data.cpu().numpy()).sum()
                    corrects += correct
                dynet.train()
                accs_test.append(corrects/len(test_dataset)*100)
    plt.close()
    return

In [33]:
%matplotlib inline

In [34]:
def pruning_func():
    global optimizer, warmup
    optimizer = torch.optim.Adam(dynet.parameters(), lr=learning_rate)
#     optimizer = torch.optim.SGD(dynet.parameters(), lr=learning_rate, momentum=0.9)
    warmup = WarmupLR_Polynomial(optimizer, 0, len(train_loader), power=0.5)
    
    
    dynet.start_computing_significance()

    for train_x, train_y in train_loader:
        train_x, train_y = train_x.to(device), train_y.to(device)
        dynet.tree.std_loss = 0.    
        yout = dynet(train_x)
#         yout.backward(gradient=torch.ones_like(yout))
        loss = criterion(yout, train_y)
        optimizer.zero_grad()
        loss.backward(retain_graph=False)

    optimizer.zero_grad()
    dynet.finish_computing_significance()
    dynet.identify_removable_neurons(num=None,
                                 threshold_min = threshold_min,
                                 threshold_max = threshold_max)
    num_remove = dynet.decay_neuron_start(decay_steps=num_decay_steps)
    if num_remove > 0:
        print(f"pruning {num_remove} neurons.")
        
        fig = plt.figure(figsize=(10,4))
        ax = fig.add_subplot(121)
        ax2 = fig.add_subplot(122)
        
        loss_list = []
        steps_ = -1
        breakall=False
        for epoch in range(train_epoch_max):
            loss_ = []
            train_acc = 0
            train_count = 0
            for train_x, train_y in train_loader:
                train_x, train_y = train_x.to(device), train_y.to(device)
                steps_ += 1

                dynet.decay_neuron_step()
                dynet.tree.std_loss = 0.    

                yout = dynet(train_x)
                loss = criterion(yout, train_y) + dynet.tree.decay_rate_std*dynet.tree.std_loss

                loss_.append(float(loss))

                optimizer.zero_grad()
                loss.backward(retain_graph=False)
                warmup.step()
                optimizer.step()

                outputs = tnn.Logits.logit_to_index(yout.data.cpu().numpy())
                targets = train_y.data.cpu().numpy()
                correct = (outputs == targets).sum()
                train_acc += correct
                train_count += len(outputs)

                dynet.decay_neuron_step()
                
                if steps_%50 == 0 and steps_>0:
                    loss = np.mean(loss_)
                    loss_ = []
                    loss_list.append(loss)
                
                if steps_%100 == 0 and steps_>0:
                    
                    _info = f'ES: {epoch}:{steps_}, Loss:{float(loss):.3f}, Acc:{correct/len(outputs)*100:.3f}%'
#                     print(_info)
                    ax.clear()
                    out = (yout.data.cpu().numpy()>0.5).astype(int)
                    ax.plot(loss_list)
                    
                    xmin, xmax = ax.get_xlim()
                    ymin, ymax = ax.get_ylim()
                    ax.text(xmin, ymin, _info)
                    
                    ax2.clear()
                    if len(accs_all)>0:
                        acc_tr = accs_all
                        acc_te = accs_test
                        if len(acc_tr)>20: acc_tr = acc_tr[-20:]
                        if len(acc_te)>20: acc_te = acc_te[-20:]
                        ax2.plot(acc_tr, label="train")
                        ax2.plot(acc_te, label="test")
                        ax2.legend(loc="lower right")

                        ymin, ymax = ax2.get_ylim()
                        ax2.text(0, 0.1*ymin+0.9*ymax, f"TR:max{max(acc_tr):.3f} end{acc_tr[-1]:.3f}")
                        ax2.text(0, 0.2*ymin+0.8*ymax, f"TE:max{max(acc_te):.3f} end{acc_te[-1]:.3f}")

                    
                    fig.canvas.draw()
                    plt.savefig("./files/_temp_prune_plot2.png")
#                     plt.pause(0.01)
#                     print("\n")
                    
                if steps_>num_decay_steps+int(num_decay_steps/2): breakall=True
                if breakall: break
            if breakall: break
                
        if not breakall:
            accs_all.append(train_acc/train_count*100.)
            with torch.no_grad():
                corrects = 0
                dynet.eval()
                for test_x, test_y in test_loader:
                    test_x  = test_x.to(device)
                    yout = dynet.forward(test_x)
                    outputs = tnn.Logits.logit_to_index(yout.data.cpu().numpy())
                    correct = (outputs == test_y.data.cpu().numpy()).sum()
                    corrects += correct
                dynet.train()
                accs_test.append(corrects/len(test_dataset)*100)
        plt.close()
    return

In [35]:
def maintain_network():
    dynet.compute_del_neurons()
    dynet.maintain_network()
    dynet.print_network()
    return

In [36]:
def save_network_stat(loop_indx):
    stdout = sys.stdout
    s = io.StringIO(newline="")
    sys.stdout = s
    dynet.print_network()
    sys.stdout = stdout
    s.seek(0)
    # prints = s.read()
    architecture = s.getvalue()
    s.close()
    
    ### number of neurons
    count = 0
    for hr in dynet.tree.DYNAMIC_LIST:
        if hr.residual:
            count += hr.residual.hidden_dim
    
    with open("05_dynamic_CNN_log_2_9.3.txt", "a+") as f:
        if loop_indx == 0:
            ### Print the configuration at top.
            f.write(f"━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\n")
            f.write(f"━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\n")
            
            from datetime import datetime
            now = datetime.now()
            dt_string = now.strftime("%B %d, %Y @ %H:%M:%S")
            f.write(f"DateTime: {dt_string}")
            
            f.write(f"num_add_neuron :{num_add_neuron}\n add_to_remove_ratio :{dynet.tree.add_to_remove_ratio}\n")
            f.write(f"learning_rate :{learning_rate}\n num_decay_steps :{num_decay_steps}\n")
            f.write(f"threshold_max :{threshold_max}\n threshold_min :{threshold_min}\n")
            f.write(f"train_epoch_min :{train_epoch_min}\n threshold_max :{train_epoch_max}\n")
            f.write(f"━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\n")
        
        f.write(f"####################| Loop:{loop_indx} | Epoch: {len(accs_all)} \n")
        num_params = sum(p.numel() for p in dynet.parameters())
        num_trainable = sum(p.numel() for p in dynet.parameters() if p.requires_grad)
        f.write(f"| Dynamic Neurons:{count} | Total Parameters: {num_params} | Trainable Parameters: {num_trainable}\n")
        f.write(f"| Train Acc:{accs_all[-1]:.3f} | Test Acc: {accs_test[-1]:.3f}\n")
        f.write(architecture)
        f.write("\n\n")

# Set all functions and begin automated loop

In [37]:
trainer = AutoTrainer()

In [38]:
trainer.adding_func = add_neurons_func
trainer.training_func = training_network_func
trainer.pruning_func = pruning_func
trainer.maintainance_func = maintain_network
trainer.log_func = save_network_stat

In [39]:
# add_neurons_func()

In [40]:
dynet.print_network()

3
╚╗
 ╚╗
  ╚╗
   ╚╗
    ╚╗
     8
    ╔╝
    8
   ╔╝
   16
  ╔╝
  32
 ╔╝
 32
╔╝
│
10


In [41]:
# for i, p in enumerate(list(dynet.parameters())):
#     print(i, p.data.shape)

In [42]:
# p = list(dynet.parameters())[3]
# print(p.shape)
# p[8:][0]

In [43]:
# p = list(dynet.parameters())[4]
# print(p.shape)
# p[0][8:]

In [44]:
# p = list(dynet.parameters())[7]
# print(p.shape)
# p

In [45]:
# dynet.print_network()

In [46]:
# dynet.root_net.residual.fc0.residual.fc1.add_hidden_neuron(2)

In [47]:
# dynet.print_network()

In [48]:
# dynet.root_net.residual.fc0.residual.fc1.residual.fc0.shortcut.weight.data.shape

In [49]:
# dynet.root_net.residual.fc0.residual.fc1.residual.fc1.shortcut.weight.data.shape

In [50]:
# dynet.root_net.residual.fc0.residual.fc1.residual.fc0.shortcut.weight

In [51]:
# dynet.root_net.residual.fc0.residual.fc1.residual.fc1.shortcut.weight

In [52]:
# dynet.root_net.residual.fc0.add_hidden_neuron(3)

In [53]:
# dynet.print_network()

In [54]:
# dynet.root_net.residual.fc0.residual.fc1.residual.fc0.shortcut.weight.data.shape

In [55]:
# dynet.root_net.residual.fc0.residual.fc1.residual.fc0.shortcut.weight

In [56]:
# dynet.root_net.residual.fc0.residual.fc0.residual.fc1.shortcut.weight.data.shape

In [57]:
# dynet.root_net.residual.fc0.residual.fc0.residual.fc1.shortcut.weight[32:]

In [58]:
# dynet.root_net.residual.fc0.residual.fc1.shortcut.weight.data.shape

In [59]:
# dynet.root_net.residual.fc0.residual.fc1.shortcut.weight[:, 32:]

In [60]:
## LOG
# adding neuron on the hr_conv layer is okay.
# adding new neuron on a hr_connector layer, makes outgoing weights zero; and incoming weights non-zero

In [61]:
# tx, ty = iter(train_loader).next()
# tx, ty = tx.to(device), ty.to(device)
# yy = dynet(tx)
# criterion(yy, ty).backward()

In [ ]:
trainer.loop(40)

Adding 56 Neurons
3
╚╗
 ╚╗
  ╚╗
   ╚╗
    ╚╗
     15
     ╠════╗
     ║    9
     ╠════╝
    ╔╝
    12
    ╠════╗
    ║    5
    ╠════╝
   ╔╝
   19
   ╠════╗
   ║    9
   ╠════╝
  ╔╝
  33
  ╠════╗
  ║    8
  ╠════╝
 ╔╝
 42
╔╝
│
10
Significance Stat:
Min, Max: (0.0, 7.0068583488464355)
Mean, Std: (1.0, 1.1059379577636719)
remove_below 0.30925285816192627
Significance:
tensor([93.9589, 93.2424, 81.4630, 87.5843, 86.7019, 97.0075, 75.2271, 51.2063,
        93.3809], device='cuda:1')
Prune:
tensor([False, False, False, False, False, False, False,  True, False],
       device='cuda:1')
Significance:
tensor([102.3470,  86.7170, 154.5639, 143.3364, 186.1050, 131.5760, 169.7406,
         63.9263, 130.3700], device='cuda:1')
Prune:
tensor([False, False, False, False, False, False,  True,  True, False],
       device='cuda:1')
Significance:
tensor([ 213.3415,  262.2608,  122.9744,  416.5245,  175.0338,  408.7890,
         667.5245,  199.7718,  316.6879,  472.0445,  203.2314,   87.7233,
         

3
╚╗
 ╚╗
  ╚╗
   ╚╗
    ╚╗
     15
     ╠════╗
     ║    13
     ╠════╝
    ╔╝
    12
    ╠════╗
    ║    5
    ╠════╝
   ╔╝
   20
   ╠════╗
   ║    9
   ╠════╝
  ╔╝
  32
  ╠════╗
  ║    9
  ╠════╝
 ╔╝
 18
╔╝
│
10
===LOOPS FINISHED :1 ===
Pausing for 2 second to give user time to STOP PROCESS
Adding 59 Neurons
3
╚╗
 ╚╗
  ╚╗
   ╚╗
    ╚╗
     22
     ╠════╗
     ║    23
     ╠════╝
    ╔╝
    15
    ╠════╗
    ║    13
    ╠════╝
   ╔╝
   27
   ╠════╗
   ║    13
   ╠════╝
  ╔╝
  37
  ╠════╗
  ║    19
  ╠════╝
 ╔╝
 23
╔╝
│
10
Significance Stat:
Min, Max: (0.0, 10.429962158203125)
Mean, Std: (0.9999999403953552, 1.544403076171875)
remove_below 0.01
Significance:
tensor([142.6819,  95.7907,  97.1135,  56.4335,  84.8902, 104.3425,  92.8710,
        152.7958,  65.3272,   0.0000,   0.0000,   0.0000,   0.0000],
       device='cuda:1')
Prune:
tensor([False, False, False, False, False, False, False, False, False,  True,
         True,  True,  True], device='cuda:1')
Significance:
tensor([108.0182

3
╚╗
 ╚╗
  ╚╗
   ╚╗
    ╚╗
     16
     ╠════╗
     ║    ╠════╗
     ║    ║    2
     ║    ╠════╝
     ║    19
     ║    ╠════╗
     ║    ║    1
     ║    ╠════╝
     ╠════╝
    ╔╝
    18
    ╠════╗
    ║    7
    ╠════╝
   ╔╝
   23
   ╠════╗
   ║    9
   ╠════╝
  ╔╝
  32
  ╠════╗
  ║    8
  ╠════╝
 ╔╝
 18
╔╝
│
10
===LOOPS FINISHED :3 ===
Pausing for 2 second to give user time to STOP PROCESS
Adding 60 Neurons
3
╚╗
 ╚╗
  ╚╗
   ╚╗
    ╚╗
     21
     ╠════╗
     ║    ╠════╗
     ║    ║    7
     ║    ╠════╝
     ║    25
     ║    ╠════╗
     ║    ║    3
     ║    ╠════╝
     ╠════╝
    ╔╝
    22
    ╠════╗
    ║    18
    ╠════╝
   ╔╝
   29
   ╠════╗
   ║    13
   ╠════╝
  ╔╝
  39
  ╠════╗
  ║    10
  ╠════╝
 ╔╝
 26
╔╝
│
10
Significance Stat:
Min, Max: (0.0, 19.983375549316406)
Mean, Std: (1.0, 2.1115057468414307)
remove_below 0.01
Significance:
tensor([102.2043, 109.9413, 131.2099,  89.3107,  97.9165, 117.6590, 106.7042,
        192.7233,  86.1856,   0.0000,   0.0000,   0.0000,   0.000

3
╚╗
 ╚╗
  ╚╗
   ╚╗
    ╚╗
     16
     ╠════╗
     ║    ╠════╗
     ║    ║    3
     ║    ╠════╝
     ║    20
     ║    ╠════╗
     ║    ║    1
     ║    ╠════╝
     ╠════╝
    ╔╝
    20
    ╠════╗
    ║    7
    ╠════╝
   ╔╝
   31
   ╠════╗
   ║    9
   ╠════╝
  ╔╝
  34
  ╠════╗
  ║    8
  ╠════╝
 ╔╝
 16
╔╝
│
10
===LOOPS FINISHED :5 ===
Pausing for 2 second to give user time to STOP PROCESS
Adding 61 Neurons
3
╚╗
 ╚╗
  ╚╗
   ╚╗
    ╚╗
     21
     ╠════╗
     ║    ╠════╗
     ║    ║    8
     ║    ╠════╝
     ║    25
     ║    ╠════╗
     ║    ║    5
     ║    ╠════╝
     ╠════╝
    ╔╝
    28
    ╠════╗
    ║    13
    ╠════╝
   ╔╝
   43
   ╠════╗
   ║    13
   ╠════╝
  ╔╝
  37
  ╠════╗
  ║    14
  ╠════╝
 ╔╝
 19
╔╝
│
10
Significance Stat:
Min, Max: (0.0, 19.06827163696289)
Mean, Std: (1.0, 2.093602180480957)
remove_below 0.01
Significance:
tensor([ 75.6052,  83.9695, 152.2096, 106.2608,  96.0002, 120.7795,  79.0409,
        164.9705, 173.2279,   0.0000,   0.0000,   0.0000,  48.5100]

3
╚╗
 ╚╗
  ╚╗
   ╚╗
    ╚╗
     13
     ╠════╗
     ║    ╠════╗
     ║    ║    4
     ║    ╠════╝
     ║    23
     ║    ╠════╗
     ║    ║    3
     ║    ╠════╝
     ╠════╝
    ╔╝
    27
    ╠════╗
    ║    7
    ╠════╝
   ╔╝
   24
   ╠════╗
   ║    10
   ╠════╝
  ╔╝
  31
  ╠════╗
  ║    8
  ╠════╝
 ╔╝
 16
╔╝
│
10
===LOOPS FINISHED :7 ===
Pausing for 2 second to give user time to STOP PROCESS
Adding 61 Neurons
3
╚╗
 ╚╗
  ╚╗
   ╚╗
    ╚╗
     17
     ╠════╗
     ║    ╠════╗
     ║    ║    14
     ║    ╠════╝
     ║    33
     ║    ╠════╗
     ║    ║    5
     ║    ╠════╝
     ╠════╝
    ╔╝
    39
    ╠════╗
    ║    11
    ╠════╝
   ╔╝
   29
   ╠════╗
   ║    15
   ╠════╝
  ╔╝
  34
  ╠════╗
  ║    9
  ╠════╝
 ╔╝
 21
╔╝
│
10
Significance Stat:
Min, Max: (0.0, 22.557880401611328)
Mean, Std: (0.9999999403953552, 2.2036283016204834)
remove_below 0.01
Significance:
tensor([112.3475,  93.9263, 141.7354, 143.5968,  92.1721, 117.9649,  94.9520,
        206.7144, 136.6927,  69.5966,   0.0000,  

3
╚╗
 ╚╗
  ╚╗
   ╚╗
    ╚╗
     15
     ╠════╗
     ║    ╠════╗
     ║    ║    4
     ║    ╠════╝
     ║    25
     ║    ╠════╗
     ║    ║    3
     ║    ╠════╝
     ╠════╝
    ╔╝
    32
    ╠════╗
    ║    8
    ╠════╝
   ╔╝
   20
   ╠════╗
   ║    9
   ╠════╝
  ╔╝
  31
  ╠════╗
  ║    7
  ╠════╝
 ╔╝
 15
╔╝
│
10
===LOOPS FINISHED :9 ===
Pausing for 2 second to give user time to STOP PROCESS
Adding 61 Neurons
3
╚╗
 ╚╗
  ╚╗
   ╚╗
    ╚╗
     22
     ╠════╗
     ║    ╠════╗
     ║    ║    8
     ║    ╠════╝
     ║    36
     ║    ╠════╗
     ║    ║    5
     ║    ╠════╝
     ╠════╝
    ╔╝
    42
    ╠════╗
    ║    12
    ╠════╝
   ╔╝
   25
   ╠════╗
   ║    12
   ╠════╝
  ╔╝
  37
  ╠════╗
  ║    13
  ╠════╝
 ╔╝
 18
╔╝
│
10
Significance Stat:
Min, Max: (0.0, 17.77925682067871)
Mean, Std: (0.9999999403953552, 2.096022605895996)
remove_below 0.01
Significance:
tensor([ 90.6244,  64.3105, 161.3813, 149.8152, 105.4896,  93.6739, 222.1402,
        139.2642,  81.2016,   0.0000,   0.0000,   0.

3
╚╗
 ╚╗
  ╚╗
   ╚╗
    ╚╗
     17
     ╠════╗
     ║    ╠════╗
     ║    ║    5
     ║    ╠════╝
     ║    25
     ║    ╠════╗
     ║    ║    4
     ║    ╠════╝
     ╠════╝
    ╔╝
    27
    ╠════╗
    ║    6
    ╠════╝
   ╔╝
   22
   ╠════╗
   ║    8
   ╠════╝
  ╔╝
  30
  ╠════╗
  ║    9
  ╠════╝
 ╔╝
 14
╔╝
│
10
===LOOPS FINISHED :11 ===
Pausing for 2 second to give user time to STOP PROCESS
Adding 61 Neurons
3
╚╗
 ╚╗
  ╚╗
   ╚╗
    ╚╗
     22
     ╠════╗
     ║    ╠════╗
     ║    ║    14
     ║    ╠════╝
     ║    34
     ║    ╠════╗
     ║    ║    13
     ║    ╠════╝
     ╠════╝
    ╔╝
    34
    ╠════╗
    ║    13
    ╠════╝
   ╔╝
   25
   ╠════╗
   ║    13
   ╠════╝
  ╔╝
  32
  ╠════╗
  ║    11
  ╠════╝
 ╔╝
 17
╔╝
│
10
Significance Stat:
Min, Max: (0.0, 20.660566329956055)
Mean, Std: (1.0, 2.1662309169769287)
remove_below 0.01
Significance:
tensor([140.7701,  50.2694, 157.0005, 141.5433, 110.2916,  93.7142, 157.9667,
         73.7391,   0.0000,   0.0000,   0.0000,  58.1273,   0.

3
╚╗
 ╚╗
  ╚╗
   ╚╗
    ╚╗
     18
     ╠════╗
     ║    ╠════╗
     ║    ║    6
     ║    ╠════╝
     ║    23
     ║    ╠════╗
     ║    ║    5
     ║    ╠════╝
     ╠════╝
    ╔╝
    24
    ╠════╗
    ║    7
    ╠════╝
   ╔╝
   24
   ╠════╗
   ║    8
   ╠════╝
  ╔╝
  25
  ╠════╗
  ║    7
  ╠════╝
 ╔╝
 13
╔╝
│
10
===LOOPS FINISHED :13 ===
Pausing for 2 second to give user time to STOP PROCESS
Adding 61 Neurons
3
╚╗
 ╚╗
  ╚╗
   ╚╗
    ╚╗
     23
     ╠════╗
     ║    ╠════╗
     ║    ║    16
     ║    ╠════╝
     ║    30
     ║    ╠════╗
     ║    ║    15
     ║    ╠════╝
     ╠════╝
    ╔╝
    26
    ╠════╗
    ║    13
    ╠════╝
   ╔╝
   31
   ╠════╗
   ║    11
   ╠════╝
  ╔╝
  27
  ╠════╗
  ║    12
  ╠════╝
 ╔╝
 17
╔╝
│
10
Significance Stat:
Min, Max: (0.0, 13.774394035339355)
Mean, Std: (1.0, 2.0582263469696045)
remove_below 0.01
Significance:
tensor([111.1837, 174.8285, 157.2640,  88.7119, 129.9240, 171.4053, 107.7543,
         97.0204,   0.0000,   0.0000,   0.0000], device='cuda:

3
╚╗
 ╚╗
  ╚╗
   ╚╗
    ╚╗
     14
     ╠════╗
     ║    ╠════╗
     ║    ║    8
     ║    ╠════╝
     ║    26
     ║    ╠════╗
     ║    ║    5
     ║    ╠════╝
     ╠════╝
    ╔╝
    21
    ╠════╗
    ║    7
    ╠════╝
   ╔╝
   18
   ╠════╗
   ║    8
   ╠════╝
  ╔╝
  26
  ╠════╗
  ║    8
  ╠════╝
 ╔╝
 12
╔╝
│
10
===LOOPS FINISHED :15 ===
Pausing for 2 second to give user time to STOP PROCESS
Adding 60 Neurons
3
╚╗
 ╚╗
  ╚╗
   ╚╗
    ╚╗
     19
     ╠════╗
     ║    ╠════╗
     ║    ║    20
     ║    ╠════╝
     ║    41
     ║    ╠════╗
     ║    ║    8
     ║    ╠════╝
     ╠════╝
    ╔╝
    24
    ╠════╗
    ║    10
    ╠════╝
   ╔╝
   20
   ╠════╗
   ║    11
   ╠════╝
  ╔╝
  31
  ╠════╗
  ║    14
  ╠════╝
 ╔╝
 15
╔╝
│
10
Significance Stat:
Min, Max: (0.0, 13.671238899230957)
Mean, Std: (1.0, 2.0171220302581787)
remove_below 0.01
Significance:
tensor([7.5602e+01, 1.6226e+02, 1.3389e+02, 1.2159e+02, 1.7914e+02, 1.0454e+02,
        7.3399e+01, 7.8792e+01, 4.6738e-05, 0.0000e+00, 0.000

Adding 60 Neurons
3
╚╗
 ╚╗
  ╚╗
   ╚╗
    ╚╗
     23
     ╠════╗
     ║    ╠════╗
     ║    ║    15
     ║    ╠════╝
     ║    31
     ║    ╠════╗
     ║    ║    11
     ║    ╠════╝
     ╠════╝
    ╔╝
    22
    ╠════╗
    ║    19
    ╠════╝
   ╔╝
   27
   ╠════╗
   ║    11
   ╠════╝
  ╔╝
  28
  ╠════╗
  ║    10
  ╠════╝
 ╔╝
 16
╔╝
│
10


In [ ]:
dynet.tree.beta_del_neuron

In [ ]:
dynet.print_network()

In [ ]:
(torch.randn(10)*10).abs().pow(0.25)

In [ ]:
# plt.plot(loss_all)

In [ ]:
plt.plot(accs_all, label="train")
plt.plot(accs_test, label="test")
ymin, ymax = plt.gca().get_ylim()
plt.text(0, 0.8*ymin+0.2*ymax, f"Train-> max:{max(accs_all):.3f} end:{accs_all[-1]:.3f} \nTest-> max:{max(accs_test):.3f} end:{accs_test[-1]:.3f}")
                    
plt.legend()
plt.savefig("files/05_dycnn_cifar10_2_9.3.png")

In [ ]:
max(accs_test)

In [ ]:
max(accs_all)

In [ ]:
np.argmax(accs_test)

In [ ]:
len(accs_test)

In [ ]:
dynet.non_linearity.bias

In [ ]:
dynet.root_net.residual.fc0.residual.fc1.residual.activations